# Building RAG with Qwen2.5

In [ ]:
import json
import os
import shutil
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# Step 0: Clear the Chroma database if it exists
persist_dir = "./chroma.db"
if os.path.exists(persist_dir):
    shutil.rmtree(persist_dir)

# Step 1: Folder for JSON Files
input_folder = "Vietnam-Law-rag_json"

# Step 2: Load All JSON Files and Convert to LangChain Documents
documents = []
for file_name in os.listdir(input_folder):
    if file_name.endswith(".json"):
        file_path = os.path.join(input_folder, file_name)
        base_file_name = os.path.splitext(file_name)[0]  # Remove the extension for `file_id`
        
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)
        
        # Convert JSON data to LangChain Document objects
        documents.extend([
            Document(
                page_content=entry["text"],
                metadata={
                    "id": entry["id"],
                    "article": entry["article"],
                    "clause": entry["clause"],
                    "title": entry["title"],
                    "file_id": base_file_name
                }
            )
            for entry in data
        ])

print(f"Loaded {len(documents)} documents from {input_folder}.")

# Step 3: Initialize HuggingFace Embeddings
embeddings_model = HuggingFaceEmbeddings()

# Step 4: Create Chroma Vector Store
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=persist_dir
)

print("Chroma database created and saved at:", persist_dir)

# Test query
## RAG database builded on cloud servers, fetch them then run the below cell

The aim is to optimize the returned data after the query search before push into the LLM Models, below here use Qwen2.5 for example.

Just download the chroma.db, then symlink or put them in the current working git folder, then run the second cell.

### CELL 2

In [2]:
import torch
from langchain_chroma import Chroma  # Use the updated Chroma import
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain_huggingface import HuggingFaceEmbeddings

# Step 1: Load the Chroma Database
persist_dir = "./chroma.db"

# Initialize the embedding function
embeddings_model = HuggingFaceEmbeddings()

# Load the Chroma database with the embedding function
vectorstore = Chroma(
    persist_directory=persist_dir,
    embedding_function=embeddings_model
)

print("Chroma database loaded.")

# Step 2: Load Qwen Model
model_id = "Qwen/Qwen2.5-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# Step 3: Set Device for GPU/CPU
device = 0 if torch.cuda.is_available() else -1

# Step 4: Create a Text-Generation Pipeline with GPU/CPU
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100,
    device=device,
    clean_up_tokenization_spaces=True
)

# Wrap the pipeline for LangChain
hf = HuggingFacePipeline(pipeline=pipe)

print("Model and pipeline initialized.")

# Move model to CPU to release GPU memory
torch.cuda.empty_cache()
model.to("cpu")
torch.cuda.empty_cache()

Chroma database loaded.
Model and pipeline initialized.


### CELL 3

In [16]:
import re
from langchain.chains import RetrievalQA

# Move model to GPU
torch.cuda.empty_cache()
model.to("cuda" if torch.cuda.is_available() else "cpu")

# Increase retrieval limit
retriever = vectorstore.as_retriever(search_kwargs={"k": 250})

qa_chain = RetrievalQA.from_chain_type(
    llm=hf,
    retriever=retriever,
    return_source_documents=True
)

# Query and retrieval
query = "Thời hạn chuẩn bị xét xử phúc thẩm là bao lâu"
result = qa_chain({"query": query})

# Print the Result
print("Answer:", result["result"])

# Print the Source Documents
print("Source Documents:")
for doc in result["source_documents"]:
    print(f"Metadata: {doc.metadata}")
    print(f"Content: {doc.page_content}\n")

# Move model to CPU to release GPU memory
torch.cuda.empty_cache()
model.to("cpu")
torch.cuda.empty_cache()

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Bảo vệ, xử lý khi công trình xảy ra sự cố hoặc có nguy cơ xảy ra sự cố.

Bộ phận phục hồi chức năng của cơ sở bảo trợ xã hội;

Kéo dài thời hạn áp dụng biện pháp xử lý hành chính.

Thực hiện bảo quản tài sản trong thời gian chờ xử lý.

Xử phạt vi phạm hành chính về quản lý thuế.

Chỉ huy xử lý các sự cố khẩn cấp hoặc bất thường xảy ra trên đường sắt;

Trục xuất, khi không áp dụng là hình phạt chính.

Cảnh báo cho dân chúng xung quanh nơi xảy ra sự cố;

Tòa án xét xử kịp thời trong thời hạn do Bộ luật này quy định, bảo đảm công bằng.

Các biện pháp thay thế xử lý vi phạm hành chính đối với người chưa thành niên bao gồm:

Tổng kết thực tiễn xét xử của các Tòa án, bảo đảm áp dụng thống nhất pháp luật trong xét xử.

Tổ chức lại bộ máy quản lý, sáp nhập hoặc chia tách bộ phận sản xuất;

Thuế tự vệ được áp dụng 

In [15]:
import re
from langchain.chains import RetrievalQA
import torch

# Move model to GPU
torch.cuda.empty_cache()
model.to("cuda" if torch.cuda.is_available() else "cpu")

# Define a custom retriever function that intercepts and modifies the results
def custom_retriever(query, k=300):
    # Retrieve documents using the retriever (use get_relevant_documents method)
    results = retriever.get_relevant_documents(query)  # Use the appropriate method
    
    modified_documents = []
    
    for doc in results:
        # Example of cleaning content: Removing unwanted words or patterns
        cleaned_content = re.sub(r"unwanted_pattern", "", doc.page_content)  # Adjust regex as needed
        
        # Optionally modify metadata (if required)
        # doc.metadata["modified"] = True
        
        # Create a new document with the modified content and metadata
        modified_doc = {
            "metadata": doc.metadata,
            "page_content": cleaned_content,
        }
        
        modified_documents.append(modified_doc)
    
    return modified_documents

# Create a custom retrieval function that uses the custom retriever to fetch and modify documents
def get_custom_documents(query):
    return custom_retriever(query)

# Create the custom retriever to pass it into the QA chain
retriever = vectorstore.as_retriever(search_kwargs={"k": 250})

# Create a custom RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=hf,  # Assuming 'hf' is your pre-trained model
    retriever=retriever,
    return_source_documents=True
)

# Query and retrieval
query = "Trẻ em là gì?"

# Use the custom retriever to retrieve and modify documents
retrieved_docs = get_custom_documents(query)

# Now, the modified documents are ready for the LLM input
input_to_llm = " ".join([doc["page_content"] for doc in retrieved_docs])

# Send the modified input to your LLM (e.g., Qwen2.5 model)
result_from_llm = hf(input_to_llm)  # Assuming 'hf' is your model callable
print("LLM Output:", result_from_llm)

# Optionally print the modified documents
print("Modified Documents:")
for doc in retrieved_docs:
    print(f"Metadata: {doc['metadata']}")
    print(f"Content: {doc['page_content']}\n")

# Move model to CPU to release GPU memory
torch.cuda.empty_cache()
model.to("cpu")
torch.cuda.empty_cache()

LLM Output: Trẻ em bị bỏ rơi; Trẻ em. Trạm y tế; Trẻ em bị mua bán; Quản trị rủi ro; Trẻ em mồ côi cả cha và mẹ; Tạm giữ tàu biển; Cơ quan quản lý nhà nước về trẻ em; Cơ quan quản lý nhà nước về trẻ em; Cơ quan quản lý nhà nước về trẻ em; Cơ quan quản lý nhà nước về trẻ em; Cơ quan quản lý nhà nước về trẻ em; Có giáo lý, giáo luật, lễ nghi; Cướp biển, cướp có vũ trang; Trẻ em vi phạm pháp luật; Cơ sở dữ liệu về con người; Trẻ em bị bóc lột; Trẻ em mồ côi cả cha và mẹ, trẻ em bị bỏ rơi, trẻ em không nơi nương tựa. Trẻ em nghiện ma túy; Y tế, văn hóa, giáo dục; Viện pháp y quốc gia thuộc Bộ Y tế; Quê quán. Trẻ em của cơ sở giáo dục mầm non; Quê quán; Quê quán; Quê quán; Trẻ em là người dưới 16 tuổi. Cơ sở y tế; Diễn biến của quá trình hòa giải; Tạm giữ tài sản, giấy tờ; Duy trì mối liên hệ giữa trẻ em với gia đình; Có giáo lý, giáo luật; Cơ sở y tế. Cơ sở y tế. Quản chế; Cột cây số; Trẻ em khuyết tật; Biện pháp sơ cứu về y tế; Bị can có nghĩa vụ: Cơ sở giáo dục. Người bị tạm giữ, người b